### SEC Edgar Form 13 Processing ###

In [ ]:
'''
What is SEC EDGAR? 
EDGAR (Electronic Data Gathering, Analysis, and Retrieval) is an online public database from the U.S. Securities and Exchange Commission (SEC). EDGAR performs automated collection, validation, indexing, acceptance, and forwarding of submissions by companies and others who are required by law to file forms with the SEC.

How is data stored at the website in general? 
Read https://www.sec.gov/developer and https://www.sec.gov/os/accessing-edgar-data pages in detail to understand how data is stored and can be easily accessed for free by anyone.

What is form 13F? 
All institutional investment managers with investment discretion of over $100 million or more must report its holdings quarterly on Form 13F with the SEC. It has information about their equity holdings. You may get detailed information about it at https://www.sec.gov/about/forms/form13f.pdf location. Additional information is available at https://www.sec.gov/divisions/investment/13ffaq.htm site.

Lookup the filings manually.
You can get to https://www.sec.gov/edgar/searchedgar/companysearch.html site and lookup information for any company or fund. Let us take an example of Berkshire Hathaway Inc. You will see it opened a site with the URL as https://www.sec.gov/edgar/browse/?CIK=1067983&owner=exclude where you can now click ‘View Filings’. You may further filter by typing 13F in the search table. If you go to the bottom of the page, you will see ‘Data source: CIK0001067983.json’. This is the json that we will use to process in this program.

Warnings This is not intended to be any form of advice. These programs are just for fun and feel free to use it anyway you like to. There is no assurance that this is always going to generate accurate information.
'''

In [ ]:
'''
High Level Logic 
Read CIK 
Lookup json for that CIK 
Parse it to filter for form 13F filings and up to last X years (in the program took oldest date as Jan 1, 2015).
The based on the adsh number, get the index.json for that filing. 
Extract the primary xml and the infotable xml files. 
Append to the dataframe and finally write to a CSV file. 
This file may then be used for analysis to see which stocks are being bought/sold
'''

In [1]:
# Get all the imports
from datetime import datetime
import numpy as np
import pandas as pd
import requests
import json
import xmltodict

In [2]:
# Change this for specific insitituition you are interested in
cik = 1067983       # Berkshire Hathaway Inc
str_cik = str(cik)
for ignore in range(0,10-len(str_cik)): # the url needs it to be in the format for 10 digits prefixed by 0 where it is 
    str_cik = '0' + str_cik
filings_url = 'https://data.sec.gov/submissions/CIK' + str_cik + '.json'
print(filings_url) # This is more for debugging / understanding

https://data.sec.gov/submissions/CIK0001067983.json


In [3]:
filings_url_json_text = requests.get(filings_url).text   # Get text from URL
print(filings_url_json_text)  # This is needed. Sometimes, we get an error from the site. Usually it works in 1-2 tries

{"cik":"1067983","entityType":"operating","sic":"6331","sicDescription":"Fire, Marine & Casualty Insurance","insiderTransactionForOwnerExists":1,"insiderTransactionForIssuerExists":1,"name":"BERKSHIRE HATHAWAY INC","tickers":["BRK-A","BRK-B"],"exchanges":["NYSE","NYSE"],"ein":"470813844","description":"","website":"","investorWebsite":"","category":"Large accelerated filer","fiscalYearEnd":"1231","stateOfIncorporation":"DE","stateOfIncorporationDescription":"DE","addresses":{"mailing":{"street1":"3555 FARNAM STREET","street2":null,"city":"OMAHA","stateOrCountry":"NE","zipCode":"68131","stateOrCountryDescription":"NE"},"business":{"street1":"3555 FARNAM STREET","street2":null,"city":"OMAHA","stateOrCountry":"NE","zipCode":"68131","stateOrCountryDescription":"NE"}},"phone":"4023461400","flags":"","formerNames":[{"name":"NBH INC","from":"1998-08-10T00:00:00.000Z","to":"1999-01-05T00:00:00.000Z"}],"filings":{"recent":{"accessionNumber":["0001193125-21-169046","0001193125-21-168049","000095

In [4]:
filings_dict = json.loads(filings_url_json_text)  # The json data is loaded into a dictionary
filings_dict.keys()      # This is more for debugging / understanding

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [6]:
base_dict = {'cik': int(filings_dict.get('cik')), 'name': filings_dict.get('name') } # use this to store common info
print('Base dict', base_dict)  # This is more for debugging / understanding

Base dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC'}


In [7]:
filings = filings_dict.get('filings').get('recent')  # get the filings
print(filings)   # This is more for debugging / understanding

{'accessionNumber': ['0001193125-21-169046', '0001193125-21-168049', '0000950123-21-007024', '0001728451-21-000003', '0001193125-21-151913', '0001209191-21-029522', '0001564590-21-022703', '0001140361-21-012994', '0001728451-21-000002', '0001193125-21-117638', '0001209191-21-026362', '0001104659-21-049166', '0001193125-21-111503', '0001387131-21-004283', '0001193125-21-109462', '0001193125-21-097370', '0001214659-21-003387', '0001623632-21-000401', '0001193125-21-080418', '0001193125-21-063192', '0001564590-21-009611', '0001193125-21-044988', '0001193125-21-044980', '0001193125-21-044970', '0001193125-21-044956', '0001193125-21-044948', '0001193125-21-044943', '0001193125-21-044938', '0001193125-21-044928', '0001193125-21-044921', '0000950123-21-002786', '0000950123-21-002781', '0001193125-21-044836', '0001193125-21-044816', '0001193125-21-044814', '0001193125-21-044798', '0001193125-21-044362', '0000093751-21-000372', '0001104659-21-017202', '0000876661-21-000178', '0001193125-21-0314

In [8]:
form13_type = ['13F-HR', '13F-HR/A', '13F-NT', '13F-NT/A']  # List of valid form types for Form 13. See specs
filings_df = pd.DataFrame(filings) # Load to dataframe -> easier to filter
filings_df.head() # This is more for debugging / understanding

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001193125-21-169046,2021-05-21,,2021-05-21T14:11:58.000Z,34,SD,001-14905,21948296,,31737,0,0,d439744dsd.htm,SD
1,0001193125-21-168049,2021-05-21,2020-12-31,2021-05-20T18:53:29.000Z,34,11-K,001-14905,21946021,,174266,0,0,d215186d11k.htm,11-K
2,0000950123-21-007024,2021-05-17,2021-03-31,2021-05-17T16:01:20.000Z,34,13F-HR,028-04545,21930169,,73179,0,0,xslForm13F_X01/primary_doc.xml,
3,0001728451-21-000003,2021-05-06,2021-05-04,2021-05-06T18:18:11.000Z,,4,,,,10483,0,0,xslF345X03/primary_doc.xml,PRIMARY DOCUMENT
4,0001193125-21-151913,2021-05-05,2021-05-01,2021-05-05T17:16:00.000Z,34,8-K,001-14905,21894557,"2.02,5.07,9.01",318305,1,1,d240122d8k.htm,8-K


In [9]:
form13fhr_df = filings_df[filings_df['form'].isin(form13_type)]  # Create dataframe and filter only form 13F

In [10]:
form13fhr_df['filingDate'] = pd.to_datetime(form13fhr_df['filingDate']) # Gives warning. Convert to datetime for comparison

<ipython-input-10-a42f0e51ca75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  form13fhr_df['filingDate'] = pd.to_datetime(form13fhr_df['filingDate']) # Gives warning. Convert to datetime for comparison


In [11]:
form13fhr_df.dtypes   # This is more for debugging. We converted the date format for filing date

accessionNumber                  object
filingDate               datetime64[ns]
reportDate                       object
acceptanceDateTime               object
act                              object
form                             object
fileNumber                       object
filmNumber                       object
items                            object
size                              int64
isXBRL                            int64
isInlineXBRL                      int64
primaryDocument                  object
primaryDocDescription            object
dtype: object

In [12]:
form13fhr_df = form13fhr_df.sort_values(by=['filingDate'], ascending=False)  # Sort for latest filing first
form13fhr_df = form13fhr_df.reset_index(drop=True)   # reseting the index
form13fhr_df.shape      # see how many records are there to be processed

(56, 14)

In [13]:
earliest_date = np.datetime64('2015-01-01')   # We will ignore older filings
form13fhr_df = form13fhr_df[form13fhr_df['filingDate'] > earliest_date]  # filter based on date set above
form13fhr_df

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000950123-21-007024,2021-05-17,2021-03-31,2021-05-17T16:01:20.000Z,34,13F-HR,028-04545,21930169,,73179,0,0,xslForm13F_X01/primary_doc.xml,
1,0000950123-21-002786,2021-02-16,2020-12-31,2021-02-16T16:06:57.000Z,34,13F-HR,028-04545,21638171,,77753,0,0,xslForm13F_X01/primary_doc.xml,
2,0000950123-21-002781,2021-02-16,2020-09-30,2021-02-16T16:05:50.000Z,34,13F-HR/A,028-04545,21638079,,6048,0,0,xslForm13F_X01/primary_doc.xml,
3,0000950123-20-012127,2020-11-16,2020-09-30,2020-11-16T16:00:35.000Z,34,13F-HR,028-04545,201316532,,79771,0,0,xslForm13F_X01/primary_doc.xml,
4,0000950123-20-009058,2020-08-14,2020-06-30,2020-08-14T16:01:47.000Z,34,13F-HR,028-04545,201104871,,76335,0,0,xslForm13F_X01/primary_doc.xml,
5,0000950123-20-005345,2020-05-15,2020-03-31,2020-05-15T16:06:46.000Z,34,13F-HR,028-04545,20885055,,101159,0,0,xslForm13F_X01/primary_doc.xml,
6,0000950123-20-002466,2020-02-14,2019-12-31,2020-02-14T16:06:35.000Z,34,13F-HR,028-04545,20619517,,102813,0,0,xslForm13F_X01/primary_doc.xml,
7,0000950123-19-011362,2019-11-14,2019-09-30,2019-11-14T16:06:50.000Z,34,13F-HR,028-04545,191220083,,100080,0,0,xslForm13F_X01/primary_doc.xml,
8,0000950123-19-008356,2019-08-14,2019-06-30,2019-08-14T16:15:46.000Z,34,13F-HR,028-04545,191026560,,97329,0,0,xslForm13F_X01/primary_doc.xml,
9,0000950123-19-005436,2019-05-15,2019-03-31,2019-05-15T16:15:14.000Z,34,13F-HR,028-04545,19828320,,98434,0,0,xslForm13F_X01/primary_doc.xml,


In [14]:
# This is recommended to use. Please replace with your own email id
headers = {
    'User-Agent': 'MyCompanyName myemailid@domain.com',
    'Referer': 'https://www.sec.gov/',
    'Accept-Encoding': 'gzip, deflate',
    'Accept': 'application/json',
    'Host': 'www.sec.gov',
    'Accept-Language': 'en-US'
}

In [15]:
# Processing each Accession Number
def process_submission_record(primary_dict):
    try:
        submission_url = 'https://www.sec.gov/Archives/edgar/data/' + str(primary_dict.get('cik')) + '/' + primary_dict.get('adsh') + '/index.json'
        print('Submission url',submission_url) # This is for debugging
        submission_url_response = requests.get(submission_url, headers=headers)
        submission_url_data = submission_url_response.text
        submission_url_dict = json.loads(submission_url_data)
        print('Submission url dict',submission_url_dict) # This is more for debugging / understanding

        files = submission_url_dict.get('directory').get('item')
        infotable_file = ''
        for file in files:
            name = file.get('name')
            if name.endswith('.xml'):
                if name == 'primary_doc.xml':    # So far every CIK appears to have this as primary doc
                    print('Primary', submission_url.replace('index.json', name))
                    primary_dict = process_primary_xml(submission_url.replace('index.json', name), primary_dict)
                else:
                    infotable_file = name     # Some has infotable.xml while others have a different name
        print('Info', infotable_file)
        return process_infotable_xml(submission_url.replace('index.json', infotable_file), primary_dict)
    except Exception as error:
        print('Got an exception in process_each_submission for ', error)


In [16]:
# Small utility function to convert string to datetime
def get_datetime(date_str):
    return datetime.strptime(date_str, '%m-%d-%Y')

In [17]:
# Process primary xml file within each accession number
def process_primary_xml(primary_url, primary_dict):
    primary_xml_dict = xmltodict.parse(requests.get(primary_url, headers=headers).text) # Read data from url to dict

    # Parse the xml
    edgar_submission_dict = primary_xml_dict.get('edgarSubmission')
    submission_cik = int(edgar_submission_dict.get('headerData').get('filerInfo').get('filer').get('credentials').get('cik'))
    if(submission_cik != primary_dict.get('cik')):   # This should never happen
        print('WARNING: Something is amiss. Primary CIK:', primary_dict.get('cik'), ' Submission CIK:', submission_cik)
    # Build other common elements for this accession number
    primary_dict['form'] = edgar_submission_dict.get('headerData').get('submissionType')
    primary_dict['report_period'] = get_datetime(edgar_submission_dict.get('headerData').get('filerInfo').get('periodOfReport'))
    primary_dict['report_calendar'] = get_datetime(edgar_submission_dict.get('formData').get('coverPage').get('reportCalendarOrQuarter'))
    primary_dict['filing_manager_name'] = edgar_submission_dict.get('formData').get('coverPage').get('filingManager').get('name')
    primary_dict['report_type'] = edgar_submission_dict.get('formData').get('coverPage').get('reportType')
    print('Primary dict', primary_dict)
    return primary_dict


In [18]:
# Process the detiled holdings file within each accession number
def process_infotable_xml(infotable_url, primary_dict):
    stocks_list = []
    infotable_xml_dict = xmltodict.parse(requests.get(infotable_url, headers=headers).text) # Read data from url to dict
    infotable_list = infotable_xml_dict.get('informationTable').get('infoTable')

    if (isinstance(infotable_list, dict)):    # Usually happens if there is only one element
        print('Single item in info table', infotable_list)
        stocks_list.append(extract_infotable_record(infotable_list, primary_dict))
    else:
        for infotable in infotable_list:   # This is where most of the times the logic should flow to
            print('Infotable element', infotable)
            stocks_list.append(extract_infotable_record(infotable, primary_dict))

    return stocks_list




In [21]:
# Extract relevant details for each share from the infotable file
def extract_infotable_record(infotable, primary_dict):
    stock_details = primary_dict.copy()
    stock_details['name_of_issuer'] = infotable.get('nameOfIssuer')
    stock_details['title_of_class'] = infotable.get('titleOfClass')
    stock_details['cusip'] = infotable.get('cusip')
    stock_details['ssh_prn_amt'] = infotable.get('value')
    stock_details['ssh_prnamt_type'] = infotable.get('sshPrnamtType')
    stock_details['investment_discretion'] = infotable.get('investmentDiscretion')
    print('Stock details', stock_details)
    return stock_details

In [22]:
# For each accession number identifed earlier, process the url to extract stocks and store in a list

complete_list = []   # List to store all records

for adsh in form13fhr_df['accessionNumber']:
    print('Processing for accession number', adsh) # This is more for debugging
    primary_dict = base_dict.copy()    # Common components for all dataframe records
    primary_dict['adsh'] = adsh.replace('-', '') # Do not need the dashes for any processing purposes
    stock_list = process_submission_record(primary_dict)   # Process each adsh record
    complete_list.extend(stock_list)  # Add records to master list
    


Processing for accession number 0000950123-21-007024
Submission url https://www.sec.gov/Archives/edgar/data/1067983/000095012321007024/index.json
Submission url dict {'directory': {'item': [{'last-modified': '2021-05-17 16:01:20', 'name': '0000950123-21-007024-index-headers.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2021-05-17 16:01:20', 'name': '0000950123-21-007024-index.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2021-05-17 16:01:20', 'name': '0000950123-21-007024.txt', 'type': 'text.gif', 'size': ''}, {'last-modified': '2021-05-17 16:01:20', 'name': '0000950123-21-007024-4471.xml', 'type': 'text.gif', 'size': '66105'}, {'last-modified': '2021-05-17 16:01:20', 'name': 'primary_doc.xml', 'type': 'text.gif', 'size': '5737'}], 'name': '/Archives/edgar/data/1067983/000095012321007024', 'parent-dir': '/Archives/edgar/data/1067983'}}
Primary https://www.sec.gov/Archives/edgar/data/1067983/000095012321007024/primary_doc.xml
Primary dict {'cik': 1067983, 'name

Stock details {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012321007024', 'form': '13F-HR', 'report_period': datetime.datetime(2021, 3, 31, 0, 0), 'report_calendar': datetime.datetime(2021, 3, 31, 0, 0), 'filing_manager_name': 'Berkshire Hathaway Inc', 'report_type': '13F COMBINATION REPORT', 'name_of_issuer': 'APPLE INC', 'title_of_class': 'COM', 'cusip': '037833100', 'ssh_prn_amt': '2963359', 'ssh_prnamt_type': None, 'investment_discretion': 'DFND'}
Infotable element OrderedDict([('nameOfIssuer', 'APPLE INC'), ('titleOfClass', 'COM'), ('cusip', '037833100'), ('value', '7106125'), ('shrsOrPrnAmt', OrderedDict([('sshPrnamt', '58175400'), ('sshPrnamtType', 'SH')])), ('investmentDiscretion', 'DFND'), ('otherManager', '4,5'), ('votingAuthority', OrderedDict([('Sole', '58175400'), ('Shared', '0'), ('None', '0')]))])
Stock details {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012321007024', 'form': '13F-HR', 'report_period': datetime.datetime(2021, 3

Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012321002786', 'form': '13F-HR', 'report_period': datetime.datetime(2020, 12, 31, 0, 0), 'report_calendar': datetime.datetime(2020, 12, 31, 0, 0), 'filing_manager_name': 'Berkshire Hathaway Inc', 'report_type': '13F COMBINATION REPORT'}
Info 0000950123-21-002786-3286.xml
Infotable element OrderedDict([('nameOfIssuer', 'ABBVIE INC'), ('titleOfClass', 'COM'), ('cusip', '00287Y109'), ('value', '2735870'), ('shrsOrPrnAmt', OrderedDict([('sshPrnamt', '25533082'), ('sshPrnamtType', 'SH')])), ('investmentDiscretion', 'DFND'), ('otherManager', '4,11'), ('votingAuthority', OrderedDict([('Sole', '25533082'), ('Shared', '0'), ('None', '0')]))])
Stock details {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012321002786', 'form': '13F-HR', 'report_period': datetime.datetime(2020, 12, 31, 0, 0), 'report_calendar': datetime.datetime(2020, 12, 31, 0, 0), 'filing_manager_name': 'Berkshire Hathaway Inc', 're

Submission url dict {'directory': {'item': [{'last-modified': '2021-02-16 16:05:50', 'name': '0000950123-21-002781-index-headers.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2021-02-16 16:05:50', 'name': '0000950123-21-002781-index.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2021-02-16 16:05:50', 'name': '0000950123-21-002781.txt', 'type': 'text.gif', 'size': ''}, {'last-modified': '2021-02-16 16:05:50', 'name': '0000950123-21-002781-3281.xml', 'type': 'text.gif', 'size': '1678'}, {'last-modified': '2021-02-16 16:05:50', 'name': 'primary_doc.xml', 'type': 'text.gif', 'size': '3027'}], 'name': '/Archives/edgar/data/1067983/000095012321002781', 'parent-dir': '/Archives/edgar/data/1067983'}}
Primary https://www.sec.gov/Archives/edgar/data/1067983/000095012321002781/primary_doc.xml
Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012321002781', 'form': '13F-HR/A', 'report_period': datetime.datetime(2020, 9, 30, 0, 0), 'report_calen

Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012320009058', 'form': '13F-HR', 'report_period': datetime.datetime(2020, 6, 30, 0, 0), 'report_calendar': datetime.datetime(2020, 6, 30, 0, 0), 'filing_manager_name': 'Berkshire Hathaway Inc', 'report_type': '13F COMBINATION REPORT'}
Info 960.xml
Infotable element OrderedDict([('nameOfIssuer', 'AMAZON COM INC'), ('titleOfClass', 'COM'), ('cusip', '023135106'), ('value', '1157325'), ('shrsOrPrnAmt', OrderedDict([('sshPrnamt', '419500'), ('sshPrnamtType', 'SH')])), ('investmentDiscretion', 'DFND'), ('otherManager', '4'), ('votingAuthority', OrderedDict([('Sole', '419500'), ('Shared', '0'), ('None', '0')]))])
Stock details {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012320009058', 'form': '13F-HR', 'report_period': datetime.datetime(2020, 6, 30, 0, 0), 'report_calendar': datetime.datetime(2020, 6, 30, 0, 0), 'filing_manager_name': 'Berkshire Hathaway Inc', 'report_type': '13F COMBINATION 

...

Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012315011992', 'form': '13F-HR', 'report_period': datetime.datetime(2015, 9, 30, 0, 0), 'report_calendar': datetime.datetime(2015, 9, 30, 0, 0), 'filing_manager_name': 'BERKSHIRE HATHAWAY INC', 'report_type': '13F COMBINATION REPORT'}
Info form13fInfoTable.xml
Infotable element OrderedDict([('nameOfIssuer', 'AT&T INC'), ('titleOfClass', 'COM'), ('cusip', '00206R102'), ('value', '415187'), ('shrsOrPrnAmt', OrderedDict([('sshPrnamt', '12743618'), ('sshPrnamtType', 'SH')])), ('investmentDiscretion', 'DFND'), ('otherManager', '4'), ('votingAuthority', OrderedDict([('Sole', '12743618'), ('Shared', '0'), ('None', '0')]))])
Stock details {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012315011992', 'form': '13F-HR', 'report_period': datetime.datetime(2015, 9, 30, 0, 0), 'report_calendar': datetime.datetime(2015, 9, 30, 0, 0), 'filing_manager_name': 'BERKSHIRE HATHAWAY INC', 'report_type': '13F CO

Submission url dict {'directory': {'item': [{'last-modified': '2015-09-04 16:01:15', 'name': '0000950123-15-009742-index-headers.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-09-04 16:01:15', 'name': '0000950123-15-009742-index.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-09-04 16:01:15', 'name': '0000950123-15-009742.txt', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-09-04 16:01:15', 'name': 'form13fInfoTable.xml', 'type': 'text.gif', 'size': '1259'}, {'last-modified': '2015-09-04 16:01:15', 'name': 'primary_doc.xml', 'type': 'text.gif', 'size': '3212'}], 'name': '/Archives/edgar/data/1067983/000095012315009742', 'parent-dir': '/Archives/edgar/data/1067983'}}
Primary https://www.sec.gov/Archives/edgar/data/1067983/000095012315009742/primary_doc.xml
Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012315009742', 'form': '13F-HR/A', 'report_period': datetime.datetime(2015, 6, 30, 0, 0), 'report_calendar': dat

Submission url dict {'directory': {'item': [{'last-modified': '2015-05-15 16:01:47', 'name': '0000950123-15-006438-index-headers.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-05-15 16:01:47', 'name': '0000950123-15-006438-index.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-05-15 16:01:47', 'name': '0000950123-15-006438.txt', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-05-15 16:01:47', 'name': 'form13fInfoTable.xml', 'type': 'text.gif', 'size': '83226'}, {'last-modified': '2015-05-15 16:01:47', 'name': 'primary_doc.xml', 'type': 'text.gif', 'size': '5883'}], 'name': '/Archives/edgar/data/1067983/000095012315006438', 'parent-dir': '/Archives/edgar/data/1067983'}}
Primary https://www.sec.gov/Archives/edgar/data/1067983/000095012315006438/primary_doc.xml
Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012315006438', 'form': '13F-HR', 'report_period': datetime.datetime(2015, 3, 31, 0, 0), 'report_calendar': date

Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012315002961', 'form': '13F-HR', 'report_period': datetime.datetime(2014, 12, 31, 0, 0), 'report_calendar': datetime.datetime(2014, 12, 31, 0, 0), 'filing_manager_name': 'Berkshire Hathaway Inc', 'report_type': '13F COMBINATION REPORT'}
Info form13fInfoTable.xml
Infotable element OrderedDict([('nameOfIssuer', 'AMERICAN EXPRESS CO'), ('titleOfClass', 'COM'), ('cusip', '025816109'), ('value', '181627'), ('shrsOrPrnAmt', OrderedDict([('sshPrnamt', '1952142'), ('sshPrnamtType', 'SH')])), ('investmentDiscretion', 'DFND'), ('otherManager', '4'), ('votingAuthority', OrderedDict([('Sole', '1952142'), ('Shared', '0'), ('None', '0')]))])
Stock details {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012315002961', 'form': '13F-HR', 'report_period': datetime.datetime(2014, 12, 31, 0, 0), 'report_calendar': datetime.datetime(2014, 12, 31, 0, 0), 'filing_manager_name': 'Berkshire Hathaway Inc', 'report_t

Submission url dict {'directory': {'item': [{'last-modified': '2015-02-17 16:30:19', 'name': '0000950123-15-002964-index-headers.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-02-17 16:30:19', 'name': '0000950123-15-002964-index.html', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-02-17 16:30:19', 'name': '0000950123-15-002964.txt', 'type': 'text.gif', 'size': ''}, {'last-modified': '2015-02-17 16:30:19', 'name': 'form13fInfoTable.xml', 'type': 'text.gif', 'size': '1813'}, {'last-modified': '2015-02-17 16:30:19', 'name': 'primary_doc.xml', 'type': 'text.gif', 'size': '3450'}], 'name': '/Archives/edgar/data/1067983/000095012315002964', 'parent-dir': '/Archives/edgar/data/1067983'}}
Primary https://www.sec.gov/Archives/edgar/data/1067983/000095012315002964/primary_doc.xml
Primary dict {'cik': 1067983, 'name': 'BERKSHIRE HATHAWAY INC', 'adsh': '000095012315002964', 'form': '13F-HR/A', 'report_period': datetime.datetime(2014, 9, 30, 0, 0), 'report_calendar': dat

In [23]:
complete_list_df = pd.DataFrame(complete_list)  # Convert the list to a dataframe. Can be used for further analysis
complete_list_df.to_excel('form13F.xlsx', sheet_name=str(cik), index=False)  # Write to excel. Can be to any output.


In [24]:
complete_list_df

,cik,name,adsh,form,report_period,report_calendar,filing_manager_name,report_type,name_of_issuer,title_of_class,cusip,ssh_prn_amt,ssh_prnamt_type,investment_discretion
0,1067983,BERKSHIRE HATHAWAY INC,000095012321007024,13F-HR,2021-03-31,2021-03-31,Berkshire Hathaway Inc,13F COMBINATION REPORT,ABBVIE INC,COM,00287Y109,2474794,None,DFND
1,1067983,BERKSHIRE HATHAWAY INC,000095012321007024,13F-HR,2021-03-31,2021-03-31,Berkshire Hathaway Inc,13F COMBINATION REPORT,AMAZON COM INC,COM,023135106,1297967,None,DFND
2,1067983,BERKSHIRE HATHAWAY INC,000095012321007024,13F-HR,2021-03-31,2021-03-31,Berkshire Hathaway Inc,13F COMBINATION REPORT,AMAZON COM INC,COM,023135106,352106,None,DFND
3,1067983,BERKSHIRE HATHAWAY INC,000095012321007024,13F-HR,2021-03-31,2021-03-31,Berkshire Hathaway Inc,13F COMBINATION REPORT,AMERICAN EXPRESS CO,COM,025816109,276111,None,DFND
4,1067983,BERKSHIRE HATHAWAY INC,000095012321007024,13F-HR,2021-03-31,2021-03-31,Berkshire Hathaway Inc,13F COMBINATION REPORT,AMERICAN EXPRESS CO,COM,025816109,23751,None,DFND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,1067983,BERKSHIRE HATHAWAY INC,000095012315002961,13F-HR,2014-12-31,2014-12-31,Berkshire Hathaway Inc,13F COMBINATION REPORT,LIBERTY GLOBAL PLC,SHS CL A,G5480U104,372861,None,DFND
3954,1067983,BERKSHIRE HATHAWAY INC,000095012315002961,13F-HR,2014-12-31,2014-12-31,Berkshire Hathaway Inc,13F COMBINATION REPORT,LIBERTY GLOBAL PLC,SHS CL C,G5480U120,354932,None,DFND
3955,1067983,BERKSHIRE HATHAWAY INC,000095012315002964,13F-HR/A,2014-09-30,2014-09-30,Berkshire Hathaway Inc,13F COMBINATION REPORT,DEERE & CO,COM,244199105,136620,None,DFND
3956,1067983,BERKSHIRE HATHAWAY INC,000095012315002964,13F-HR/A,2014-09-30,2014-09-30,Berkshire Hathaway Inc,13F COMBINATION REPORT,DEERE & CO,COM,244199105,189599,None,DFND
